In [1]:
##arxiv -- to interact with all research papers in its website
##Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun 
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper 


In [ ]:
##Used inbuilt tool pf wikipedia

api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [4]:
##Used arxiv  tool
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [5]:
tools=[wiki,arxiv]


In [7]:
##create custom tools[RAG TOOL]
from langchain_community.document_loaders import WebBaseLoader 
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [8]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documnets=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectorstoredb=FAISS.from_documents(documnets,OpenAIEmbeddings())
retriever=vectorstoredb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C078B1E290>, search_kwargs={})

In [9]:
##to use retriever as tool,convert i bto tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","search any information about lansmith")
retriever_tool.name

'langsmith-search'

In [ ]:
tools=[wiki,arxiv,retriever_tool]


In [18]:
from dotenv import load_dotenv
load_dotenv()
import os
import openai
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Llama3-8b-8192")

In [ ]:
##Run all this tools with agents and llm models
##Tools,LLM-->Agent executor




In [19]:
##Prompt
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")

In [20]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [21]:
##USe agent to excute all tools,llms,prompts together use agent
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt) 
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [22]:
# AGent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [23]:
agent_executor.invoke({"input":"Tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQSDK referenceLangChain off-the-shelf evaluatorsQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!
1. Install LangSmith​
PythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai
2. Create an API key​
To create an API key head to the Settings page. Then click Create

{'input': 'Tell me about langsmith',
 'output': 'I was able to get more information about LangSmith using the tool. It appears to be a platform for building production-grade LLM (Large Language Model) applications. It allows users to closely monitor and evaluate their applications, making it possible to ship quickly and with confidence.\n\nAccording to the information provided, LangSmith can be installed using Python or TypeScript, and an API key needs to be created. The platform also provides a way to set up the environment, log the first trace, and run the first evaluation.\n\nAdditionally, LangSmith seems to have a community aspect, with a Discord channel, Twitter handle, and GitHub repository. There are also code examples and documentation available for Python and JavaScript.\n\nPlease let me know if you would like me to continue using tools to gather more information or if you would like me to provide a direct response based on the information gathered so far.'}